In [1]:
import open3d as o3d
import numpy as np

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


# Define functions

In [10]:
def visualize_point_cloud(nome_point_cloud):
    pcd = o3d.io.read_point_cloud(nome_point_cloud, format = 'ply')
    print(np.asarray(pcd.points))
    o3d.visualization.draw_geometries([pcd])
    return pcd

In [13]:
def display_inlier_outlier(cloud, ind):
    inlier_cloud = cloud.select_by_index(ind)
    outlier_cloud = cloud.select_by_index(ind, invert = True)

    #print("Showing outliers (red) and inliers (gray): ")
    #outlier_cloud.paint_uniform_color([1, 0, 0])
    #inlier_cloud.paint_uniform_color([0.8, 0.8, 0.8])
    o3d.visualization.draw_geometries([inlier_cloud])
    #print("inlier_cloud: ", inlier_cloud)

In [14]:
def save_point_cloud(nome_file, punti_utili, colori_utili):
    file = open(nome_file,"w")
    file.write("ply \n")
    file.write("format ascii 1.0 \n")
    file.write("element vertex " + str(punti_utili.shape[0]) + "\n")
    file.write("property float x \n")
    file.write("property float y \n")
    file.write("property float z \n")
    file.write("property uchar red \n")
    file.write("property uchar green \n")
    file.write("property uchar blue \n")
    file.write("end_header \n")
    i = 0
    j = 0
    x = 0
    for i in range(np.shape(punti_utili)[0]):
        for j in range (3):
            file.write(str(punti_utili[i, j]) + " ")
        for x in range (3):
            file.write(str(colori_utili[i, x]) + " ")
        file.write("\n")

In [15]:
#Modificare il valore di nb_neighbors per specificare quanti vicini devono essere presi in considerazione per calcolare la distanza media per un dato punto
#Modificare il valore di std_ratio in base a quanto aggressiva si vuole la funzione, rispettivamente più è basso il valore più outlier verranno eliminati 
def statistical_outlier_removal(pcd, nb_neighbors, std_ratio):
    print("Statistical outlier removal")
    cl, ind = pcd.remove_statistical_outlier(nb_neighbors, std_ratio)
    display_inlier_outlier(pcd, ind)
    return cl

In [16]:
#Modificare il valore di nb_points per selezionare la quantità minima di punti che la sfera deve contenere
#Modificare il valore di radius per definire il raggio della sfera che verrà utilizzata per il conteggio dei vicini
def radius_outlier_removal(pcd, nb_points, radius):
    print("Radius outlier removal")
    cl, ind = pcd.remove_radius_outlier(nb_points, radius)
    display_inlier_outlier(pcd, ind)
    return cl

# Point cloud cleaning

In [7]:
pcd = visualize_point_cloud("bata.ply")

[[ 0.817455  -0.14911    1.43862  ]
 [ 0.997424  -0.340987   1.73491  ]
 [ 0.0961574  0.385165   1.4249   ]
 ...
 [ 0.0507136  0.188813   1.98618  ]
 [ 0.168587   0.0949471  1.61455  ]
 [ 0.0633312  0.199371   1.32529  ]]
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: Handle non valido. 


In [29]:
min_x = -2.0
max_x = 2.0
min_y = -3.0
max_y = 3.0
min_z = -3.0
max_z = 3.0
#filtra per posizione in modo tale da restringere il campo di interesse della point cloud
filtered_pcd = np.where((np.asarray(pcd.points)[:,0] < max_x) & (np.asarray(pcd.points)[:,0] > min_x) & 
                        (np.asarray(pcd.points)[:,1] < max_y) & (np.asarray(pcd.points)[:,1] > min_y) &
                        (np.asarray(pcd.points)[:,2] < max_z) & (np.asarray(pcd.points)[:,1] > min_z))
print(filtered_pcd)
print(np.shape(filtered_pcd))

(array([     0,      1,      2, ..., 342702, 342703, 342704], dtype=int64),)
(1, 272697)


In [30]:
punti_utili = np.zeros((np.shape(filtered_pcd)[1], 3), dtype = float)
colori_utili = np.zeros((np.shape(filtered_pcd)[1], 3), dtype = float)
print(np.shape(punti_utili))
print(np.shape(colori_utili))

(272697, 3)
(272697, 3)


In [31]:
#filtra i punti e li salva in una nuova matrice
i = 0
j = 0
x = 0
for i in range(np.shape(np.asarray(pcd.points))[0]):
    if ((np.asarray(pcd.points)[i,0] < max_x) & (np.asarray(pcd.points)[i,0] > min_x) & 
        (np.asarray(pcd.points)[i,1] < max_y) & (np.asarray(pcd.points)[i,1] > min_y) &
        (np.asarray(pcd.points)[i,2] < max_z) & (np.asarray(pcd.points)[i,1] > min_z)):
        for j in range(3):
            punti_utili[x,j] = np.asarray(pcd.points)[i,j]
        x = x + 1
print(punti_utili)

[[ 0.817455  -0.14911    1.43862  ]
 [ 0.997424  -0.340987   1.73491  ]
 [ 0.0961574  0.385165   1.4249   ]
 ...
 [ 0.0507136  0.188813   1.98618  ]
 [ 0.168587   0.0949471  1.61455  ]
 [ 0.0633312  0.199371   1.32529  ]]


In [32]:
#crea una nuova matrice con i colori dei punti che non nel campo di interesse e li riporta in scala [0, 255]
i = 0
j = 0
x = 0
for i in range(np.shape(np.asarray(pcd.points))[0]):
    if ((np.asarray(pcd.points)[i,0] < max_x) & (np.asarray(pcd.points)[i,0] > min_x) & 
        (np.asarray(pcd.points)[i,1] < max_y) & (np.asarray(pcd.points)[i,1] > min_y) &
        (np.asarray(pcd.points)[i,2] < max_z) & (np.asarray(pcd.points)[i,1] > min_z)):
        for j in range(3):
            colori_utili[x,j] = np.asarray(pcd.colors)[i,j]*255
        x = x + 1
colori_utili = colori_utili.astype(int)
print(colori_utili)

[[  0 255   0]
 [  0 255   0]
 [  0 255   0]
 ...
 [141 166 198]
 [  0   0   0]
 [  1   0   0]]


In [33]:
save_point_cloud("pointCloud_PrimaModifica_bata.ply", punti_utili, colori_utili)

In [8]:
pcd_first_modify = visualize_point_cloud("pointCloud_PrimaModifica_bata.ply")

[[ 0.817455  -0.14911    1.43862  ]
 [ 0.997424  -0.340987   1.73491  ]
 [ 0.0961574  0.385165   1.4249   ]
 ...
 [ 0.0507136  0.188813   1.98618  ]
 [ 0.168587   0.0949471  1.61455  ]
 [ 0.0633312  0.199371   1.32529  ]]


In [35]:
# property float x
print(max(np.asarray(pcd.points)[:,0]))
print(min(np.asarray(pcd.points)[:,0]))

9.78069
-6.29391


In [36]:
# property float y
print(max(np.asarray(pcd.points)[:,1]))
print(min(np.asarray(pcd.points)[:,1]))

5.84287
-6.18714


In [37]:
# property float z
print(max(np.asarray(pcd.points)[:,2]))
print(min(np.asarray(pcd.points)[:,2]))

12.4549
-3.42999


In [38]:
green_value = 1
#elimina le telecamere che sono completamente verdi
filtered_pcd_green_less = np.where((np.asarray(pcd_first_modify.colors)[:,1] != green_value))
print(filtered_pcd_green_less)
print(np.shape(filtered_pcd_green_less))

(array([  2198,   2199,   2200, ..., 272694, 272695, 272696], dtype=int64),)
(1, 270342)


In [39]:
punti_utili_green_less = np.zeros((np.shape(filtered_pcd_green_less)[1], 3), dtype = float)
colori_utili_green_less = np.zeros((np.shape(filtered_pcd_green_less)[1], 3), dtype = float)
print(np.shape(punti_utili_green_less))
print(np.shape(colori_utili_green_less))

(270342, 3)
(270342, 3)


In [40]:
#filtra i punti e li salva in una nuova matrice
i = 0
j = 0
x = 0
for i in range(np.shape(np.asarray(pcd_first_modify.points))[0]):
    if (np.asarray(pcd_first_modify.colors)[i,1] != green_value):
        for j in range(3):
            punti_utili_green_less[x,j] = np.asarray(pcd_first_modify.points)[i,j]
        x = x + 1
print(punti_utili_green_less)

[[0.0355728 0.196954  2.01228  ]
 [0.0161714 0.16316   1.9858   ]
 [0.0683236 0.153718  1.9817   ]
 ...
 [0.0507136 0.188813  1.98618  ]
 [0.168587  0.0949471 1.61455  ]
 [0.0633312 0.199371  1.32529  ]]


In [41]:
#crea una nuova matrice con i colori dei punti che non sono telecamere e li riporta in scala [0, 255]
i = 0
j = 0
x = 0
for i in range(np.shape(np.asarray(pcd_first_modify.points))[0]):
    if (np.asarray(pcd_first_modify.colors)[i,1] != green_value):
        for j in range(3):
            colori_utili_green_less[x,j] = np.asarray(pcd_first_modify.colors)[i,j] * 255
        x = x + 1
colori_utili_green_less = colori_utili_green_less.astype(int)
print(colori_utili_green_less)

[[189 215 254]
 [ 87  77  48]
 [187 187 179]
 ...
 [141 166 198]
 [  0   0   0]
 [  1   0   0]]


In [42]:
save_point_cloud("pointCloud_SecondaModifica_bata.ply", punti_utili_green_less, colori_utili_green_less)

In [11]:
pcd_second_modify = visualize_point_cloud("pointCloud_SecondaModifica_bata.ply")

[[0.0355728 0.196954  2.01228  ]
 [0.0161714 0.16316   1.9858   ]
 [0.0683236 0.153718  1.9817   ]
 ...
 [0.0507136 0.188813  1.98618  ]
 [0.168587  0.0949471 1.61455  ]
 [0.0633312 0.199371  1.32529  ]]
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: L'operazione di trasformazione richiesta non è supportata. 


In [17]:
cl_statical = statistical_outlier_removal(pcd_second_modify, 20, 2.0)

Statistical outlier removal


In [19]:
cl_statical2 = statistical_outlier_removal(cl_statical, 20, 2.0)

Statistical outlier removal


In [22]:
cl_radius = radius_outlier_removal(cl_statical2, 16, 0.5)

Radius outlier removal


In [47]:
print(np.asarray(cl_radius.points))
print(" ")
cl_radius_colori = np.asarray(cl_radius.colors) * 255
cl_radius_colori = cl_radius_colori.astype(int)
print(cl_radius_colori)

[[0.0355728 0.196954  2.01228  ]
 [0.0161714 0.16316   1.9858   ]
 [0.0683236 0.153718  1.9817   ]
 ...
 [0.203753  0.0393841 1.91172  ]
 [0.164142  0.0331354 1.92038  ]
 [0.0507136 0.188813  1.98618  ]]
 
[[189 215 254]
 [ 87  77  48]
 [187 187 179]
 ...
 [130  99  70]
 [123  92  63]
 [141 166 198]]


In [48]:
save_point_cloud("pointCloud_TerzaModifica_bata.ply", np.asarray(cl_radius.points), cl_radius_colori)

In [ ]:
pcd_third_modify = visualize_point_cloud("pointCloud_TerzaModifica_bata.ply")

In [50]:
range_value = 0.019
#filtra i colori grigi
filtered_pcd_gray_less = np.where((np.asarray(pcd_third_modify.colors)[:,0] > ((np.asarray(pcd_third_modify.colors)[:,1]) + range_value)) | 
                                  (np.asarray(pcd_third_modify.colors)[:,0] < ((np.asarray(pcd_third_modify.colors)[:,1]) - range_value)) & 
                                  (np.asarray(pcd_third_modify.colors)[:,0] > ((np.asarray(pcd_third_modify.colors)[:,2]) + range_value)) | 
                                  (np.asarray(pcd_third_modify.colors)[:,0] < ((np.asarray(pcd_third_modify.colors)[:,2]) - range_value)))
print(filtered_pcd_gray_less)
print(np.shape(filtered_pcd_gray_less))

(array([     0,      1,      3, ..., 251740, 251741, 251742], dtype=int64),)
(1, 243162)


In [51]:
punti_utili_gray_less = np.zeros((np.shape(filtered_pcd_gray_less)[1], 3), dtype = float)
colori_utili_gray_less = np.zeros((np.shape(filtered_pcd_gray_less)[1], 3), dtype = float)
print(np.shape(punti_utili_gray_less))
print(np.shape(colori_utili_gray_less))

(243162, 3)
(243162, 3)


In [52]:
#filtra i punti e li salva in una nuova matrice
i = 0
j = 0
x = 0
for i in range(np.shape(np.asarray(pcd_third_modify.points))[0]):
    if ((np.asarray(pcd_third_modify.colors)[i,0] > ((np.asarray(pcd_third_modify.colors)[i,1]) + range_value)) | 
        (np.asarray(pcd_third_modify.colors)[i,0] < ((np.asarray(pcd_third_modify.colors)[i,1]) - range_value)) & 
        (np.asarray(pcd_third_modify.colors)[i,0] > ((np.asarray(pcd_third_modify.colors)[i,2]) + range_value)) | 
        (np.asarray(pcd_third_modify.colors)[i,0] < ((np.asarray(pcd_third_modify.colors)[i,2]) - range_value))):
        for j in range(3):
            punti_utili_gray_less[x,j] = np.asarray(pcd_third_modify.points)[i,j]
        x = x + 1
print(punti_utili_gray_less)

[[0.0355728 0.196954  2.01228  ]
 [0.0161714 0.16316   1.9858   ]
 [0.0640235 0.250512  2.05054  ]
 ...
 [0.203753  0.0393841 1.91172  ]
 [0.164142  0.0331354 1.92038  ]
 [0.0507136 0.188813  1.98618  ]]


In [53]:
#crea una nuova matrice con i colori dei punti che non superano la soglia e li riporta in scala [0, 255]
i = 0
j = 0
x = 0
for i in range(np.shape(np.asarray(pcd_third_modify.points))[0]):
    if ((np.asarray(pcd_third_modify.colors)[i,0] > ((np.asarray(pcd_third_modify.colors)[i,1]) + range_value)) | 
        (np.asarray(pcd_third_modify.colors)[i,0] < ((np.asarray(pcd_third_modify.colors)[i,1]) - range_value)) & 
        (np.asarray(pcd_third_modify.colors)[i,0] > ((np.asarray(pcd_third_modify.colors)[i,2]) + range_value)) | 
        (np.asarray(pcd_third_modify.colors)[i,0] < ((np.asarray(pcd_third_modify.colors)[i,2]) - range_value))):
        for j in range(3):
            colori_utili_gray_less[x,j] = np.asarray(pcd_third_modify.colors)[i,j] * 255
        x = x + 1
colori_utili_gray_less = colori_utili_gray_less.astype(int)
print(colori_utili_gray_less)

[[189 215 254]
 [ 87  77  48]
 [121  91  66]
 ...
 [130  99  70]
 [123  92  63]
 [141 166 198]]


In [54]:
save_point_cloud("pointCloud_GrayLess_bata.ply", punti_utili_gray_less, colori_utili_gray_less)

In [55]:
pcd_gray_less = visualize_point_cloud("pointCloud_GrayLess_bata.ply")

[[0.0355728 0.196954  2.01228  ]
 [0.0161714 0.16316   1.9858   ]
 [0.0640235 0.250512  2.05054  ]
 ...
 [0.203753  0.0393841 1.91172  ]
 [0.164142  0.0331354 1.92038  ]
 [0.0507136 0.188813  1.98618  ]]


In [56]:
blue_value = 0.66
#filtra i colori sul blu
filtered_pcd_blue_less = np.where(((np.asarray(pcd_gray_less.colors)[:,0]) > (np.asarray(pcd_gray_less.colors)[:,2])) &
                                  ((np.asarray(pcd_gray_less.colors)[:,1]) > (np.asarray(pcd_gray_less.colors)[:,2])) &
                                  ((np.asarray(pcd_gray_less.colors)[:,2]) < blue_value))
print(filtered_pcd_blue_less)
print(np.shape(filtered_pcd_blue_less))

(array([     1,      2,      4, ..., 243158, 243159, 243160], dtype=int64),)
(1, 199182)


In [57]:
punti_utili_blue_less = np.zeros((np.shape(filtered_pcd_blue_less)[1], 3), dtype = float)
colori_utili_blue_less = np.zeros((np.shape(filtered_pcd_blue_less)[1], 3), dtype = float)
print(np.shape(punti_utili_blue_less))
print(np.shape(colori_utili_blue_less))

(199182, 3)
(199182, 3)


In [58]:
#filtra i punti e li salva in una nuova matrice
i = 0
j = 0
x = 0
for i in range(np.shape(np.asarray(pcd_gray_less.points))[0]):
    if (((np.asarray(pcd_gray_less.colors)[i,0]) > (np.asarray(pcd_gray_less.colors)[i,2])) &
        ((np.asarray(pcd_gray_less.colors)[i,1]) > (np.asarray(pcd_gray_less.colors)[i,2])) &
        ((np.asarray(pcd_gray_less.colors)[i,2]) < blue_value)):
        for j in range(3):
            punti_utili_blue_less[x,j] = np.asarray(pcd_gray_less.points)[i,j]
        x = x + 1
print(punti_utili_blue_less)

[[ 1.61714e-02  1.63160e-01  1.98580e+00]
 [ 6.40235e-02  2.50512e-01  2.05054e+00]
 [ 2.35207e-03  1.68977e-01  1.98956e+00]
 ...
 [ 1.46329e-01 -8.90432e-04  1.94789e+00]
 [ 2.03753e-01  3.93841e-02  1.91172e+00]
 [ 1.64142e-01  3.31354e-02  1.92038e+00]]


In [59]:
#crea una nuova matrice con i colori dei punti che non superano la soglia e li riporta in scala [0, 255]
i = 0
j = 0
x = 0
for i in range(np.shape(np.asarray(pcd_gray_less.points))[0]):
    if (((np.asarray(pcd_gray_less.colors)[i,0]) > (np.asarray(pcd_gray_less.colors)[i,2])) &
        ((np.asarray(pcd_gray_less.colors)[i,1]) > (np.asarray(pcd_gray_less.colors)[i,2])) &
        ((np.asarray(pcd_gray_less.colors)[i,2]) < blue_value)):
        for j in range(3):
            colori_utili_blue_less[x,j] = np.asarray(pcd_gray_less.colors)[i,j] * 255
        x = x + 1
colori_utili_blue_less = colori_utili_blue_less.astype(int)
print(colori_utili_blue_less)

[[ 87  77  48]
 [121  91  66]
 [ 56  47  27]
 ...
 [177 154 130]
 [130  99  70]
 [123  92  63]]


In [60]:
save_point_cloud("pointCloud_BlueLess_bata.ply", punti_utili_blue_less, colori_utili_blue_less)

In [7]:
pcd_blue_less = visualize_point_cloud("pointCloud_BlueLess_bata.ply")

[[ 1.61714e-02  1.63160e-01  1.98580e+00]
 [ 6.40235e-02  2.50512e-01  2.05054e+00]
 [ 2.35207e-03  1.68977e-01  1.98956e+00]
 ...
 [ 1.46329e-01 -8.90432e-04  1.94789e+00]
 [ 2.03753e-01  3.93841e-02  1.91172e+00]
 [ 1.64142e-01  3.31354e-02  1.92038e+00]]


In [8]:
cl_statical3 = statistical_outlier_removal(pcd_blue_less, 80, 2.0)

Statistical outlier removal
Showing outliers (red) and inliers (gray): 


In [9]:
#crea una mesh dalla point cloud
alpha = 0.3

print(f"alpha = {alpha:.3f}")
mesh = o3d.geometry.TriangleMesh.create_from_point_cloud_alpha_shape(cl_statical3, alpha)
mesh.compute_vertex_normals()
o3d.visualization.draw_geometries([mesh], mesh_show_back_face = True)

alpha=0.300


In [10]:
#ricostruisce la point cloud dalla mesh
pcd_ricostruita = mesh.sample_points_uniformly(number_of_points = 200000)
o3d.visualization.draw_geometries([pcd_ricostruita])

[Open3D WARNING] [ViewControl] SetViewPoint() failed because window height and width are not set.


In [92]:
print(np.asarray(pcd_ricostruita.points))
print(" ")
pcd_ricostruita_colori = np.asarray(pcd_ricostruita.colors) * 255
pcd_ricostruita_colori = pcd_ricostruita_colori.astype(int)
print(pcd_ricostruita_colori)

[[-0.07653915  0.02364968  1.84749406]
 [-0.07688607  0.03924939  1.84939751]
 [-0.06394922  0.03220384  1.84837935]
 ...
 [ 0.01863712 -0.03769238  2.52227686]
 [ 0.0135616  -0.07560691  2.56669333]
 [ 0.01624577 -0.04919757  2.534356  ]]
 
[[163 137 117]
 [172 146 125]
 [188 160 138]
 ...
 [106  77  49]
 [107  77  50]
 [104  76  48]]


In [93]:
save_point_cloud("pointCloud_Ricostruita_bata.ply", np.asarray(pcd_ricostruita.points), pcd_ricostruita_colori)

In [8]:
pcd_finale = visualize_point_cloud("pointCloud_Ricostruita_bata.ply")

[[-0.07653915  0.02364968  1.84749406]
 [-0.07688607  0.03924939  1.84939751]
 [-0.06394922  0.03220384  1.84837935]
 ...
 [ 0.01863712 -0.03769238  2.52227686]
 [ 0.0135616  -0.07560691  2.56669333]
 [ 0.01624577 -0.04919757  2.534356  ]]
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: L'operazione di trasformazione richiesta non è supportata. 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: Handle non valido. 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: L'operazione di trasformazione richiesta non è supportata. 


# Error estimate calculation

In [ ]:
#salva la mesh
o3d.io.write_triangle_mesh("mesh_bata.ply", mesh)

In [2]:
#carica la mesh
mesh_ruotata_bata = o3d.io.read_triangle_mesh("BataRuotata.ply")
o3d.visualization.draw_geometries([mesh_ruotata_bata], mesh_show_back_face = True)

In [3]:
#ricostruisce la point cloud dalla mesh
pcd_ricostruita_ruotata_bata = mesh_ruotata_bata.sample_points_uniformly(number_of_points = 200000)
o3d.visualization.draw_geometries([pcd_ricostruita_ruotata_bata])

In [4]:
#carica la mesh
mesh_scatola_bata = o3d.io.read_triangle_mesh("ScatolaBata.ply")
o3d.visualization.draw_geometries([mesh_scatola_bata], mesh_show_back_face = True)

In [5]:
#ricostruisce la point cloud dalla mesh
pcd_scatola_bata = mesh_scatola_bata.sample_points_uniformly(number_of_points = 200000)
o3d.visualization.draw_geometries([pcd_scatola_bata])

In [6]:
#calcola la distanza da una nuvola di punti di origine a una nuvola di punti di destinazione 
#calcola per ogni punto nella nuvola di punti di origine la distanza dal punto più vicino nella nuvola di punti di destinazione
dists_bata = pcd_ricostruita_ruotata_bata.compute_point_cloud_distance(pcd_scatola_bata)
dists_bata = np.asarray(dists_bata)
print(dists_bata)

[0.07522647 0.0749466  0.07586065 ... 0.043552   0.04423099 0.04980978]


In [7]:
np.mean(dists_bata)

0.05401846822566649